<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Set-cookies" data-toc-modified-id="Set-cookies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Set cookies</a></span></li><li><span><a href="#Function-to-get-firm-uid" data-toc-modified-id="Function-to-get-firm-uid-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Function to get firm uid</a></span></li><li><span><a href="#Functions-to-get-base-information" data-toc-modified-id="Functions-to-get-base-information-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Functions to get base information</a></span></li><li><span><a href="#Get-base-info-and-save" data-toc-modified-id="Get-base-info-and-save-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Get base info and save</a></span></li><li><span><a href="#Get-adv-info-and-save" data-toc-modified-id="Get-adv-info-and-save-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Get adv info and save</a></span></li><li><span><a href="#Combine-base-and-adv-info" data-toc-modified-id="Combine-base-and-adv-info-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Combine base and adv info</a></span></li><li><span><a href="#Write-into-excel-file" data-toc-modified-id="Write-into-excel-file-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Write into excel file</a></span></li></ul></div>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bs4 import BeautifulSoup
from urllib import parse
import requests
from tqdm import tqdm_notebook
from time import sleep
import pandas as pd
import openpyxl
from utils import *

ModuleNotFoundError: No module named 'utils'

## Set cookies

In [3]:
cookie = "QCCSESSID=r18mmd4r1lvs24cca9t43969b5; zg_did=%7B%22did%22%3A%20%2216bc94370d11b4-047d80b89838b1-37677e02-384000-16bc94370d2471%22%7D; UM_distinctid=16bc94371142a0-021542bc12af83-37677e02-384000-16bc9437115489; hasShow=1; _uab_collina=156244976879017749455047; acw_tc=2ff6129815624497682152240ef19f61e0ffef38ea7b331d445687c36c; Hm_lvt_3456bee468c83cc63fb5147f119f1075=1562449769; CNZZDATA1254842228=1120220450-1562445588-https%253A%252F%252Fwww.qichacha.com%252F%7C1562488799; Hm_lpvt_3456bee468c83cc63fb5147f119f1075=1562492933; zg_de1d1a35bfa24ce29bbf2c7eb17e6c4f=%7B%22sid%22%3A%201562486598400%2C%22updated%22%3A%201562492960763%2C%22info%22%3A%201562449768671%2C%22superProperty%22%3A%20%22%7B%7D%22%2C%22platform%22%3A%20%22%7B%7D%22%2C%22utm%22%3A%20%22%7B%7D%22%2C%22referrerDomain%22%3A%20%22%22%2C%22cuid%22%3A%20%22af5c198de42103292f80181258ee27a7%22%7D"

## Function to get firm uid

In [ ]:
index_names_df = pd.read_csv('../index_name.csv', sep='\t', header=None,  names=['index', 'name'])
len(index_names_df)

In [ ]:
# define the header which includes cookies
header_uids = {
    "Host": "www.qichacha.com",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36",
    "Accept": "text/html, */*; q=0.01",
    "Accept-Language": "en,zh-CN;q=0.9,zh;q=0.8,zh-HK;q=0.7",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.qichacha.com/",
    "Cookie": cookie,
    "Connection": "keep-alive",
    "X-Requested-With": "XMLHttpRequest"
}

In [ ]:
uids_df = get_firm_uid(header=header_uids, index_names_df=index_names_df)
len(uids_df)

In [ ]:
# merge two dataframe
index_name_uid_df = pd.merge(left=index_names_df, right=uids_df, how='inner', on='index')
index_name_uid_df.head()

## Functions to get base information

In [ ]:
# create headers list
header_list = []
for uid in list(index_name_uid_df['uid']):
    header_list.append({
    "Host": "www.qichacha.com",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36",
    "Accept": "text/html, */*; q=0.01",
    "Accept-Language": "en,zh-CN;q=0.9,zh;q=0.8,zh-HK;q=0.7",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.qichacha.com/firm_{}.html".format(uid),
    "Cookie": cookie,
    "Connection": "keep-alive",
    "X-Requested-With": "XMLHttpRequest"
})

## Get base info and save

In [ ]:
soup_base_info_list = []
for header, uid in tqdm_notebook(zip(header_list, \
                                     list(index_name_uid_df['uid'])), \
                                 total=len(header_list)):
    response_info = requests.get("https://www.qichacha.com/firm_{}.html#history".format(uid), headers=header)
    soup_base_info = BeautifulSoup(response_info.content)
    # check the length of response, if access denied, i.e. request is limited
    if len(soup_base_info) == 1:
        raise Exception("Access denied")
    soup_base_info_list.append(soup_base_info)
    sleep(5)
    
base_info_df = get_firm_base_info(soup_base_info_list)

## Get adv info and save

In [ ]:
names = list(base_info_df['name'])
soup_adv_info_list = []
for header, uid, name in tqdm_notebook(zip(header_list, \
                                           list(index_name_uid_df['uid']), names), \
                                       total=len(header_list)):
    response_info = requests.get("https://www.qichacha.com/company_getinfos?unique={}&companyname={}&tab=report".format(uid, parse.quote(name)), headers=header)
    soup_adv_info = BeautifulSoup(response_info.content)
    # check the length of response, if access denied, i.e. request is limited
    if len(response_info.content) == 1:
        raise Exception("Access denied")
    soup_adv_info_list.append(soup_adv_info)
adv_info_df = get_firm_adv_info(soup_adv_info_list)

## Combine base and adv info

In [ ]:
df = pd.concat([index_name_uid_df, \
                base_info_df.drop(columns=['name']), \
                adv_info_df], axis=1, sort=False)

In [ ]:
# add new columns: firm info site page
df['firm_info_site'] = df['uid'].map(lambda x: \
                                     "https://www.qichacha.com/firm_{}.html".format(x))
# save df to xlsx
df.to_excel('../all_df.xlsx')
# read from xlsx
# df = pd.read_excel('../all_df.xlsx', index_col=0)

## Write into excel file

In [ ]:
to_excel_file('../sample.xlsx', df)